In [9]:
#-*- coding:utf-8 -*-
import os
import sys
import json
import operator
from flask import Flask,g
import time
import pandas as pd
import numpy as np
from flask import jsonify
import json
import inspect
import redis
from flask import request

In [6]:
ip = 'localhost'
reload(sys)
sys.setdefaultencoding('utf8')

app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False
r = redis.Redis(host=ip, port=6379, db=0)

In [23]:
table1 = pd.read_excel("TAG_LOG_DOWNLOAD.xlsx")
table1['TAG_VALUE'] = ""
table2 = pd.read_excel("TAG_Value_DOWNLOAD.xlsx")
table_merge = pd.concat([table1.loc[:,:], table2.loc[:,:]], axis = 0)
table_merge = table_merge.reset_index(drop=True)

In [16]:
import psycopg2
# connect PostgreSQL
conn = psycopg2.connect(database = "MJ",
                        host = "localhost",
                        user = "llc",
                        password = "")
cur = conn.cursor()

In [14]:
def tag_db_query(UTID, sql_db):
    SQL = "SELECT tag_info FROM intent_db WHERE tag_info->>'TAG_ID' = '{}';".format(UTID)
    sql_db.execute(SQL)
    records = sql_db.fetchall()[0][0]
    return records

In [17]:
unique_ID = np.unique(table_merge['ID'])
for i in unique_ID:
    temp_list = []
    temp_detail = {}    
    unique_ID_table = table_merge[table_merge.ID==i]
    temp_detail['VID'] = i
    temp_detail['realtimeTag'] = []

    for j in range(len(unique_ID_table)):
        query_update = {}
        query_temp = tag_db_query(unique_ID_table['UTID'].values[j],cur)

        query_update['scenario'] = query_temp['scenario']
        query_update['recommendWeight'] = query_temp['recommedWeight']
        query_update['dataSource'] = query_temp['dataSource']
        query_update['securityFilter'] = query_temp['securityFilter']
        query_update['tagValue'] = unique_ID_table['TAG_VALUE'].values[j]
        query_update['tagId'] = query_temp['TAG_ID']
        query_update['tagName'] = query_temp['Chinese_Desc']
        query_update['tagTime'] = time.time()
        query_update['expireTime'] = time.time() + float(query_temp['expireAfter'])
        query_update['isValid'] = query_temp['isActive']
        temp_list.append(query_update)
    temp_detail['batchTag'] = temp_list
    r.set(i, json.dumps(temp_detail,ensure_ascii=False))

In [102]:
tag_info = json.loads(r.get(1500))

for k in range(len(tag_info.values()[0])):
    if str(tag_info.values()[0][k]['tagId']) == 'SEG002':
        new_info = tag_info.values()[0][k]
new_info

{u'dataSource': u'EBM',
 u'expireTime': 1511318698.777089,
 u'isValid': u'Y',
 u'recommendWeight': u'1',
 u'scenario': {u'DIIS1': u'S', u'DIIS2': u'SEG002', u'DIIS3': u'02'},
 u'securityFilter': u'admin',
 u'tagId': u'SEG002',
 u'tagName': u'\u5ba2\u7fa42',
 u'tagTime': 1511318668.777089,
 u'tagValue': 3}

In [89]:
tag_info = json.loads(r.get(1))

for k in range(len(tag_info.values()[0])):
    if str(tag_info.values()[0][k]['tagId']) == 'SEG002':
        new_info = tag_info.values()[0][k]['tagValue']
new_info

3

In [110]:
for k in range( len((tag_info[u'batchTag'])[0])):
    print t

In [ ]:
@app.route('/GetAllTag',methods=['GET'])
def get_alltag():
    vid = request.args.get('vid')
    tagid = request.args.get('tagid')
    
    
    tag_info = json.loads(r.get(vid))
    res = jsonify(tag_info)
    res.headers['Content-Type'] = 'application/json; charset=utf-8'
    return res



In [5]:
if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=6004)

SystemExit: 1

In [95]:
r.get(1500)

'{"batchTag": [{"tagTime": 1511318668.774434, "scenario": {"DIIS2": "SEG001", "DIIS3": "01", "DIIS1": "S"}, "isValid": "Y", "tagId": "SEG001", "securityFilter": "admin", "expireTime": 1511318698.774434, "tagValue": 7, "dataSource": "EBM", "tagName": "\xe5\xae\xa2\xe7\xbe\xa41", "recommendWeight": "1"}, {"tagTime": 1511318668.777089, "scenario": {"DIIS2": "SEG002", "DIIS3": "02", "DIIS1": "S"}, "isValid": "Y", "tagId": "SEG002", "securityFilter": "admin", "expireTime": 1511318698.777089, "tagValue": 3, "dataSource": "EBM", "tagName": "\xe5\xae\xa2\xe7\xbe\xa42", "recommendWeight": "1"}], "realtimeTag": [], "VID": 1500}'

In [111]:
tmp = {"batchTag": [{"tagTime": 1511429720.788613, "scenario": {"DIIS2": "SEG001", "DIIS3": "01", "DIIS1": "S"}, "isValid": "Y", "tagId": "SEG001", "securityFilter": "admin", "expireTime": 1511429750.788614, "tagValue": 7, "dataSource": "EBM", "tagName": "客群1", "recommendWeight": "1"}, {"tagTime": 1511429720.791564, "scenario": {"DIIS2": "SEG002", "DIIS3": "02", "DIIS1": "S"}, "isValid": "Y", "tagId": "SEG002", "securityFilter": "admin", "expireTime": 1511429750.791565, "tagValue": 3, "dataSource": "EBM", "tagName": "客群2", "recommendWeight": "1"}], "realtimeTag": [], "VID": 1500}

In [115]:
tmp["realtimeTag"] = [{u'create Time': u'17385', u'referenceDocument': u'SEG_Simulation', u'scenario': {u'DIIS2': u'SEG001', u'DIIS3': u'01', u'DIIS1': u'S'}, u'recommedWeight': u'1', u'Chinese_Desc': u'\u5ba2\u7fa41', u'TAG_ID': u'SEG001', u'operationType': u'Batch', u'securityFilter': u'admin', u'update Time': u'17385', u'expireAfter': u'30', u'tagVersion': u'V1', u'modelId': u'MSEG00001', u'isActive': u'Y', u'dataSource': u'EBM'}]

In [116]:
tmp

{'VID': 1500,
 'batchTag': [{'dataSource': 'EBM',
   'expireTime': 1511429750.788614,
   'isValid': 'Y',
   'recommendWeight': '1',
   'scenario': {'DIIS1': 'S', 'DIIS2': 'SEG001', 'DIIS3': '01'},
   'securityFilter': 'admin',
   'tagId': 'SEG001',
   'tagName': '\xe5\xae\xa2\xe7\xbe\xa41',
   'tagTime': 1511429720.788613,
   'tagValue': 7},
  {'dataSource': 'EBM',
   'expireTime': 1511429750.791565,
   'isValid': 'Y',
   'recommendWeight': '1',
   'scenario': {'DIIS1': 'S', 'DIIS2': 'SEG002', 'DIIS3': '02'},
   'securityFilter': 'admin',
   'tagId': 'SEG002',
   'tagName': '\xe5\xae\xa2\xe7\xbe\xa42',
   'tagTime': 1511429720.791564,
   'tagValue': 3}],
 'realtimeTag': [{u'Chinese_Desc': u'\u5ba2\u7fa41',
   u'TAG_ID': u'SEG001',
   u'create Time': u'17385',
   u'dataSource': u'EBM',
   u'expireAfter': u'30',
   u'isActive': u'Y',
   u'modelId': u'MSEG00001',
   u'operationType': u'Batch',
   u'recommedWeight': u'1',
   u'referenceDocument': u'SEG_Simulation',
   u'scenario': {u'DIIS1

In [129]:
tmp["realtimeTag"]

[{u'Chinese_Desc': u'\u5ba2\u7fa41',
  u'TAG_ID': u'SEG001',
  u'create Time': u'17385',
  u'dataSource': u'EBM',
  u'expireAfter': u'30',
  u'isActive': u'Y',
  u'modelId': u'MSEG00001',
  u'operationType': u'Batch',
  u'recommedWeight': u'1',
  u'referenceDocument': u'SEG_Simulation',
  u'scenario': {u'DIIS1': u'S', u'DIIS2': u'SEG001', u'DIIS3': u'01'},
  u'securityFilter': u'admin',
  u'tagVersion': u'V1',
  u'update Time': u'17385'}]